In [24]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math, time, os
from torch.utils.data import Dataset, DataLoader
import tiktoken
# from torch.cuda.amp import autocast, GradScaler
from torch.amp.autocast_mode import autocast
from torch.amp.grad_scaler import GradScaler
from tqdm import tqdm

In [25]:
from components.dataset import TextDataset
from components.model import GPTModel
from components.tokenizer import encode, decode, tokenizer

In [26]:
from datasets import load_dataset

# dataset = load_dataset("wikimedia/wikipedia", "20231101.en")
dataset = load_dataset("starhopp3r/TinyChat")
# This gives you cleaned, plain text articles1
print(dataset['train'][100]['text'][:500])  # Print the first 500 characters of the first article
print(dataset['train'][600000])

[INST] Hello, I feel a bit sad today because things seem hard to understand and move through. [/INST] I understand how you feel; sometimes life can be heavy like a thick substance we cannot lift. [INST] Yes, it can be very difficult, especially for young people trying to find their way. [/INST] Young minds often carry many questions that can weigh them down with worries and doubts. [INST] Sometimes, I wish everything would get better and we could all feel lighter again. [/INST] Hoping for better
{'text': "[INST] Do you think the disease spreading in the city is really as bad as it seems? [/INST] It does seem very clear that many people are crying over the current situation. [INST] Yes, I feel disgusted by how quickly it is spreading without control or care. [/INST] It makes me feel unwell just to think about how people's lives are affected deeply. [INST] I can’t believe some people ignore the danger and spread the disease even more. [/INST] That kind of behavior is truly unhelpful and 

In [27]:
#hyperparameters
train_model = False
block_size = 128
n_layers = 8
n_heads = 8
dropout_p = 0.1
batch_size =8
learning_rate = 3e-4
n_embedding = 128
max_iters = 5000
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [28]:
# tokenizer = tiktoken.get_encoding("gpt2")

train_dataset = TextDataset(dataset, block_size=block_size)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=16)

In [29]:
# define objects
vocab_size = tokenizer.n_vocab

model = GPTModel(vocab_size, n_embedding, n_layers, n_heads, dropout_p, block_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [30]:


# training loop
torch.set_float32_matmul_precision('high')
scaler = GradScaler(device)
if train_model:
    compiled_model = torch.compile(model)

    pbar = tqdm(range(max_iters), desc="Training", ncols=100)
    data_iter = iter(train_dataloader)

    for count in pbar:
        # xb, yb = next(data_iter)

        try:
            xb, yb = next(data_iter)
        except StopIteration:
            # dataloader exhausted — restart it
            data_iter = iter(train_dataloader)
            xb, yb = next(data_iter)
        if count%100 == 0:
          # print out xb, yb, encoded too
          print('xb decoded: ', decode(xb[0].tolist())) 
          print('yb decoded: ', decode(yb[0].tolist())) 

        # except StopIteration:
        #     break  # dataloader exhausted before max_iters
        
        xb, yb = xb.to(device), yb.to(device)
        # logits = compiled_model(xb)
        # loss = loss_fn(logits.view(-1, vocab_size), yb.view(-1))

        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        with autocast(device, dtype=torch.float16):
            logits = compiled_model(xb)
            loss = loss_fn(logits.view(-1, vocab_size), yb.view(-1))

        # backward pass with gradient scaling
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update bar text dynamically
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

In [31]:
if train_model:
  torch.save(model.state_dict(), "checkpoints/gpt_model-1.pth")
else:
  model.load_state_dict(torch.load("checkpoints/gpt_model-1.pth"))

In [32]:
@torch.no_grad()
def generate_text(model, tokenizer, prompt, max_new_tokens, block_size, device):
    model.eval()
    # Encode the prompt text into token IDs
    tokens = torch.tensor(encode(prompt), dtype=torch.long).unsqueeze(0).to(device)

    for _ in range(max_new_tokens):
        # Only keep the last block_size tokens for context
        input_tokens = tokens[:, -block_size:]

        # Get logits and take the last token’s distribution
        logits = model(input_tokens)
        logits = logits[:, -1, :]  # (batch=1, vocab)
        probs = F.softmax(logits, dim=-1)

        # Sample from the distribution
        next_token = torch.multinomial(probs, num_samples=1)
        tokens = torch.cat((tokens, next_token), dim=1)

    # Decode back into text
    output_text = tokenizer.decode(tokens[0].tolist())
    return output_text
  
# print model parameters
print (f"Model has {sum(p.numel() for p in model.parameters())/1000000} million parameters.")
prompt = "how are you doing today? [/INST]"
print(generate_text(model, tokenizer, prompt, max_new_tokens=500, block_size=block_size, device=device))

Model has 17.677395 million parameters.
how are you doing today? [/INST] talk trousers we order and torn wires [/INST] I think it is surprising to have to walk that. It feels that life. [INST] What. [INST] I wonder my some enough[INST] I try to handle every my. [/INST] It really fun, much we have. [INST] Hi. [INST] I wish it makes emotions you think there my experience today in or such pictures to find a growing steamively to enjoy to use and show. [/INST][/INST][INST] Hello, and creativity. [INST] Maybe[INST] Why sharing more[INST] I agree can really see today around, and they is a hiding can enjoy; we should go for my to explore and by parts [/INST] It makes in nature surprised. [/INST] It is a visit. [/INST] Absolutely by to the small problem; time towards upset,, pictures to, so. [/INST][INST] Hello, that balloons reflecting can help us learn about our, I world. [/INST] Yes painting and feelings about how or take can help;, reflecting of a plan. [/INST] Change is surely learn and h